In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg


from IPython.display import HTML

import os
import sys

import h5py
import numpy as np
import pandas as pd
import random as rn
import seaborn as sns
import tensorflow as tf

# Necessário mudar o diretório de trabalho para o nível mais acima
if not 'notebooks' in os.listdir('.'):
    current_dir = os.path.abspath(os.getcwd())
    parent_dir = os.path.dirname(current_dir)
    os.chdir(parent_dir)
    
from src.animation import animate_heat_map
from src.predictor_consumer import PredictionConsumer

In [2]:
seed = 32
rn.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

sns.set_context('notebook')
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette("Paired"))

tf.get_logger().setLevel('ERROR')
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

In [3]:
dataset_path = 'data/tiny-dataset.hdf5'

with h5py.File(dataset_path, 'r') as in_:
    data = in_['real'][...]

In [4]:
# Especificar dimensões
time_series = ('time_series', range(data.shape[0]))
time_step = ('time_step', range(data.shape[1]))
pos_x = ('pos_x', range(data.shape[2]))
pos_y = ('pos_y', range(data.shape[3]))

# Remover última dimensão espúria
squeezed_data = np.squeeze(data, axis=-1)

# Converter para pandas 
df = pd.DataFrame(squeezed_data.ravel(), columns=['temperatura'])

# Salvar array
output_dir = 'data'
temperatura_data_fp = os.path.join(output_dir, 'temperatura.data')
df.values.ravel().astype('float64').tofile(temperatura_data_fp)

In [5]:
tfx_host = 'localhost'
tfx_port = 8501

model_name = 'arima_22'

pc = PredictionConsumer(host=tfx_host, port=tfx_port, model_name=model_name)

In [6]:
time_series_ix = 100
x = squeezed_data[[time_series_ix], :-1]
y = squeezed_data[[time_series_ix], 1:]
y_hat = pc.predict(x)

In [7]:
anim_y = animate_heat_map(np.squeeze(y,axis=0))
anim_y_html = anim_y.to_html5_video()
anim_yhat = animate_heat_map(np.squeeze(y_hat,axis=0))
anim_yhat_html = anim_yhat.to_html5_video()
HTML(f'<div style="float: left;"> {anim_y_html} </div><div style="float: left;"> {anim_yhat_html} </div>')

In [8]:
# Inserir aqui o caminho da biblioteca do savime
py_savime_path =  '/home/daniel/PycharmProjects/intelipetro/savime'
sys.path.insert(0, py_savime_path)

In [9]:
from schema.tar import Tar, ImplicitTarDimensionSpecification, TarAttributeSpecification
from savime.datatype import SavimeSupportedTypes
from schema.schema import IntervalRange, IndexRange
from schema.dataset import FileDataset
from schema.subtar import OrderedSubTarDimensionSpecification, SubTarAttributeSpecification, SubTar
from savime.client import Client
from misc.command_runner import CommandRunner

In [10]:
num_models = 25
num_observations, num_time_steps, num_pos_x, num_pos_y = squeezed_data.shape

def define_implicit_tar_dimension(name, start, stop, step=1, data_type=SavimeSupportedTypes.INT32):
    return ImplicitTarDimensionSpecification(name, data_type, IntervalRange(start, stop, step))

def define_attribute(name, data_type=SavimeSupportedTypes.DOUBLE, num_columns=1):
    return TarAttributeSpecification(name, data_type, num_columns)

def define_tar(name, dimensions, attributes):
    return Tar(name, dimensions, attributes)

def define_ordered_sub_tar_dimension(dimension, start, stop, is_physical=False):
    return OrderedSubTarDimensionSpecification(dimension, IndexRange(start, stop, is_physical))

def define_subtar_attribute(attribute, dataset):
    return SubTarAttributeSpecification(attribute, dataset)

def define_subtar(tar, dimensions, attributes):
    return SubTar(tar, dimensions, attributes)

def define_file_dataset(name, path, data_type=SavimeSupportedTypes.DOUBLE, is_in_savime_storage=False):
    return FileDataset(name=name, file_path=path, data_type=data_type,
                       is_in_savime_storage=is_in_savime_storage)
    

###
dataset = define_file_dataset('temperature_data', temperatura_data_fp)

###
observation_dim = define_implicit_tar_dimension('observation', 1, num_observations)

time_step_dim = define_implicit_tar_dimension('time_step', 1, num_time_steps)

pos_x_dim = define_implicit_tar_dimension('pos_x', 1, num_pos_x)

pos_y_dim = define_implicit_tar_dimension('pos_y', 1, num_pos_y)

temperature = define_attribute('temperature')

dims = [observation_dim, time_step_dim, pos_x_dim, pos_y_dim]
attributes = [temperature]

tar = define_tar('temperatures_tar', dims, attributes)

###
observation_dim_sub = define_ordered_sub_tar_dimension(observation_dim, 1, num_observations)

time_step_dim_sub = define_ordered_sub_tar_dimension(time_step_dim, 1, num_time_steps)

pos_x_dim_sub = define_ordered_sub_tar_dimension(pos_x_dim, 1, num_pos_x)

pos_y_dim_sub = define_ordered_sub_tar_dimension(pos_y_dim, 1, num_pos_y)

temperature_sub = define_subtar_attribute(temperature, dataset)

subtar_dims = [observation_dim_sub, time_step_dim_sub, pos_x_dim_sub, pos_y_dim_sub]
subtar_attrs = [temperature_sub]
subtar = define_subtar(tar, subtar_dims, subtar_attrs)

In [11]:
with Client(host='127.0.0.1', port=65000, raise_silent_error=True) as client:
    command_runner = CommandRunner(client)
    command_runner.create(dataset)
    command_runner.create(tar)
    command_runner.load(subtar)

2020-02-05 18:33:46 [Connector]:DEBUG: Trying to open a connection on the port 65000.
2020-02-05 18:33:46 [Connector]:INFO: A connection has been opened on the port 65000.
2020-02-05 18:33:46 [Client]:INFO: Running the query CREATE_DATASET("temperature_data:double:1", "@data/temperatura.data");.
2020-02-05 18:33:46 [Client]:INFO: Query handler response message: Query executed successfully
2020-02-05 18:33:46 [Timer]:INFO: It took 0.000002s to run `Client.process_query_response`.
2020-02-05 18:33:46 [Timer]:INFO: It took 0.130474s to run `Client.execute`.
2020-02-05 18:33:46 [Client]:INFO: Running the query CREATE_TAR("temperatures_tar", "*", "implicit, observation, int32, 1, 400, 1 | implicit, time_step, int32, 1, 10, 1 | implicit, pos_x, int32, 1, 35, 1 | implicit, pos_y, int32, 1, 40, 1", "temperature, double: 1"  );.
2020-02-05 18:33:46 [Client]:INFO: Query handler response message: Query executed successfully
2020-02-05 18:33:46 [Timer]:INFO: It took 0.000001s to run `Client.proces

In [12]:
with Client(host='127.0.0.1', port=65000, raise_silent_error=True) as client:
    responses = client.execute(f'SELECT({tar.name})')

2020-02-05 18:33:46 [Connector]:DEBUG: Trying to open a connection on the port 65000.
2020-02-05 18:33:46 [Connector]:INFO: A connection has been opened on the port 65000.
2020-02-05 18:33:46 [Client]:INFO: Running the query SELECT(temperatures_tar).
2020-02-05 18:33:47 [Client]:INFO: Query handler response message: #observation,d,int32|time_step,d,int32|pos_x,d,int32|pos_y,d,int32|temperature,a,double:1
2020-02-05 18:33:47 [Client]:DEBUG: Trying to receive a buffer with 22400000 bytes.
2020-02-05 18:33:47 [Client]:DEBUG: Trying to receive a buffer with 22400000 bytes.
2020-02-05 18:33:47 [Client]:DEBUG: Trying to receive a buffer with 22400000 bytes.
2020-02-05 18:33:47 [Client]:DEBUG: Trying to receive a buffer with 44800000 bytes.
2020-02-05 18:33:47 [Client]:DEBUG: Trying to receive a buffer with 22400000 bytes.
2020-02-05 18:33:47 [Client]:DEBUG: Block received.
2020-02-05 18:33:47 [Client]:DEBUG: Returning an array with 5600000 rows.
2020-02-05 18:33:47 [Client]:DEBUG: Read query

In [13]:
for response in responses:
    print(response)

DataVariableBlock(dims=OrderedDict([('observation', array([[  1],
       [  1],
       [  1],
       ...,
       [400],
       [400],
       [400]], dtype=int32)), ('pos_x', array([[ 1],
       [ 1],
       [ 1],
       ...,
       [35],
       [35],
       [35]], dtype=int32)), ('pos_y', array([[ 1],
       [ 2],
       [ 3],
       ...,
       [38],
       [39],
       [40]], dtype=int32)), ('time_step', array([[ 1],
       [ 1],
       [ 1],
       ...,
       [10],
       [10],
       [10]], dtype=int32))]), attrs=OrderedDict([('temperature', array([[24.2499939 ],
       [24.7499939 ],
       [24.9500061 ],
       ...,
       [21.05001221],
       [21.14998779],
       [21.2499939 ]]))]))


In [14]:
responses[0].dims['observation'].shape

(5600000, 1)